### This file contains example using tools to evaluate the performances for color recognition category for VizWiz data.

Evaluation tools from https://github.com/GT-Vision-Lab/VQA.

Codes in eval_util.py from Yanan Wang

In [1]:
import pandas as pd
import numpy as np
from eval_util import *

In [2]:
train_features = pd.read_csv("./azure_features_images/data/vizwiz_train_color_recognition.csv", delimiter=";", engine="python")
train_features['answer'] = train_features['descriptions'].astype(str).apply(lambda x: x.lower())
train_features['QID'] = train_features['qid']

In [3]:
train_targets = pd.read_csv("../vizwiz_skill_typ_train.csv", engine="python")

In [4]:
train = pd.merge(train_features, train_targets, how='left', on='QID')

In [5]:
set(train.values[0, 15:25])

{'basil', 'basil leaves'}

In [10]:
mat[0]

array(['VizWiz_train_000000000000.jpg',
       "What's the name of this product?",
       "[{'text': 'a bottle of wine on a table', 'confidence': 0.7408009412153109}]",
       "['bottle', 'indoor', 'table', 'sitting', 'wine', 'food', 'glass', 'plate', 'red', 'sandwich', 'laying', 'coffee', 'white']",
       "['Brown']",
       "[{'text': 'a bottle of wine on a table', 'confidence': 0.7408009412153109}]",
       'VizWiz_train_000000000000.jpg', 'VizWiz_train_000000000000.jpg',
       "What's the name of this product?", 'other', 1, 1, 0, 0, 0,
       'basil leaves', 'basil leaves', 'basil', 'basil', 'basil leaves',
       'basil leaves', 'basil leaves', 'basil leaves', 'basil leaves',
       'basil'], dtype=object)

In [19]:
mat.dtype

dtype('O')

In [6]:
# current visual question answers' accuracy (agreement)
ans_accuracy  = []
# accuracy within each given answer type
type_accuracy = {}

mat = train.values

for i in range(len(mat)):
    resAns = mat[i,5]
    # clean up str
    ans_original    = re.sub('[\n\t\s]', ' ', resAns).strip()
    ans_punctuation = processPunctuation(ans_original)
    ans_final       = str(processDigitArticle(ans_punctuation))
    accuracy_scores = []
    # evaluate 10 different crowd answers
    crowd_answers = mat[i, 15:25]
    if len(set(crowd_answers)) > 1:
        for a in crowd_answers:
            a = processPunctuation(a)
    for curr in crowd_answers:
        other_ans = [ans for ans in crowd_answers if ans != curr]
        # find agreement to measure accuracy
        matches = []
        for ans in other_ans:
            ans = str(ans)
            if ans.isdigit() or ans in ["no", "yes"]:
                ans = ans.center(len(ans)+2)
            if ans in ans_final:
                matches.append(ans)
        # get VQA eval score
        score = min(1, float(len(matches)/3))
        accuracy_scores.append(score)
    # record accuracy for agreement & answer type 
    ans_type = mat[i,9]
    ans_accuracy.append(np.mean(accuracy_scores))
    if ans_type not in type_accuracy:
        type_accuracy[ans_type] = []
    type_accuracy[ans_type].append(score)
    qid = mat[i,0]
    setEvalQA(qid, score)
    setEvalAnsType(qid, ans_type, score)


setAccuracy(ans_accuracy, type_accuracy)

AttributeError: 'float' object has no attribute 'isdigit'

In [16]:
setAccuracy(accQA, accAnsType)

In [18]:
print(accuracy['overall'])         
print(accuracy['perAnswerType'])

4.74
{'yes/no': 0.65, 'other': 5.47, 'unanswerable': 2.74, 'number': 0.0}
